# Download ORACC

In [1]:
import os, sys, math
import requests
import json
import time
import random
import shutil
import zipfile
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [2]:
import oracc

In [3]:
out_dir = os.path.abspath(f"/Volumes/FrankDisk/oracc_zips")
os.makedirs(out_dir, exist_ok=True)
out_dir

'/Volumes/FrankDisk/oracc_zips'

## Download the projects

In [4]:
project_zips = oracc.get_all_project_zips(out_dir, verbose=False, tqdm=tqdm)

  0%|          | 0/136 [00:00<?, ?it/s]

In [5]:
print(len(project_zips))
project_zips[:3]

115


['/Volumes/FrankDisk/oracc_zips/adsd.zip',
 '/Volumes/FrankDisk/oracc_zips/adsd-adart1.zip',
 '/Volumes/FrankDisk/oracc_zips/adsd-adart2.zip']

## Download Translations

In [6]:
all_translated_ids = oracc.get_all_translated_object_ids(project_zips, tqdm)

  0%|          | 0/115 [00:00<?, ?it/s]

In [7]:
print(len(all_translated_ids))
all_translated_ids[1_000:1_005]

16277


[('aemw/ugarit', 'P503427'),
 ('aemw/ugarit', 'P503428'),
 ('aemw/ugarit', 'P503429'),
 ('aemw/ugarit', 'P503430'),
 ('aemw/ugarit', 'P503431')]

In [ ]:
def download_object_translation(out_dir, project_id, object_id):
    url = f"http://oracc.iaas.upenn.edu/{project_id}/{object_id}/html"
    odir = f"{out_dir}/html/{object_id[:2]}"
    out_path = f"{odir}/{object_id}.html"
    if os.path.exists(out_path):
        return out_path
    os.makedirs(odir, exist_ok=True)
    html_text = oracc.session.get(url).text
    with open(out_path, "wt") as f:
        f.write(html_text)
    return out_path
    
for pid, oid in tqdm(all_translated_ids[:]):
    tpath = download_object_translation(out_dir, pid, oid)
#     print(tpath)

  0%|          | 0/16277 [00:00<?, ?it/s]